In [18]:
import pandas as pd
import re

In [6]:
def datafram_from_csv(file_path):
    df = pd.read_csv(file_path,encoding='latin1')
    return df

In [9]:
def get_concat_data():
    list_of_files = ['data/Insta_labeled_data/image_labels_0plus_to_10_.csv','data/Insta_labeled_data/image_labels_10plus_to_40.csv','data/Insta_labeled_data/image_labels_40plus.csv']
    list_of_df = [datafram_from_csv(i) for i in list_of_files]
    df = pd.concat(list_of_df, axis=0, ignore_index=True)
    return df

In [12]:
df = get_concat_data()

In [13]:
df.head()

,id_image,label_image
0,https://scontent.cdninstagram.com/hphotos-xaf1...,Person Person\nCelebrity\nClothes\nText Pers...
1,https://scontent.cdninstagram.com/hphotos-xap1...,Person Celebrity celebritÃ Person
2,https://scontent.cdninstagram.com/hphotos-xap1...,Nature\nPerson\nClothes\nFood Person\nFood P...
3,https://scontent.cdninstagram.com/hphotos-xaf1...,Cartoon Other art dontknow
4,https://scontent.cdninstagram.com/hphotos-xap1...,Clothes Clothes Clothes


In [24]:
objects = [j for i in df['label_image'].tolist() for j in re.split('\W+', i) if j!='']

In [30]:
unique_objects = set(objects)
len(unique_objects)

568

In [32]:
other_objects = []
name_objects = []
result = sorted([(x, objects.count(x)) for x in unique_objects], key=lambda y: y[1])
for elem in result:
    print(f'{elem[0]} {elem[1]}')

Address 1
accessories 1
lighter 1
Figurine 1
body 1
campaign 1
clubbing 1
technics 1
Alcohol 1
ad 1
mame 1
DJ 1
smokey 1
name 1
bag 1
Plastic 1
celebrity 1
postcards 1
towers 1
cigar 1
buiding 1
Entradas 1
nickles 1
Place 1
weird 1
ANIMALS 1
that 1
Bowties 1
HARRY 1
full 1
hands 1
Pool 1
nascar 1
televison 1
vip 1
phones 1
wood 1
Pet 1
object 1
gey 1
journal 1
studio 1
hunting 1
dance 1
fuck 1
terror 1
education 1
cupcake 1
Paint 1
grass 1
talking 1
vegetable 1
traficc 1
Nightlife 1
glasses 1
soccer 1
club 1
night 1
article 1
bed 1
wristwatch 1
cellphone 1
Thumbs 1
Guitar 1
VILLAGE 1
possing 1
garden 1
Legs 1
book 1
puzzle 1
Ð½Ñ 1
tigar 1
edifice 1
product 1
XRAY 1
Jesus 1
Vacation 1
Smoke 1
show 1
terms 1
tweet 1
Luxury 1
sketch 1
wallet 1
crashed 1
Injured 1
corals 1
Kids 1
Fun 1
tampons 1
rubber 1
glass 1
crayons 1
plane 1
LGBT 1
FanFic 1
a 1
social 1
inappropriate 1
concussion 1
web 1
tiger 1
nude 1
cd 1
trash 1
phrases 1
landscape 1
team 1
park 1
make 1
baby 1
Broke 1
Swimming 1
i